In [1]:
import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession, DataFrame

# This CATALOG_URL works for the "docker compose" testing and development environment
# Change 'lakekeeper' if you are not running on "docker compose" (f. ex. 'localhost' if Lakekeeper is running locally).
CATALOG_URL = "http://lakekeeper:8181/catalog"
WAREHOUSE = "demo"

SPARK_VERSION = pyspark.__version__
SPARK_MINOR_VERSION = '.'.join(SPARK_VERSION.split('.')[:2])
ICEBERG_VERSION = "1.10.0"

# Configure our Spark Session

In [2]:
config = {
    "spark.sql.catalog.lakekeeper": "org.apache.iceberg.spark.SparkCatalog",
    "spark.sql.catalog.lakekeeper.type": "rest",
    "spark.sql.catalog.lakekeeper.uri": CATALOG_URL,
    "spark.sql.catalog.lakekeeper.warehouse": WAREHOUSE,
    "spark.sql.catalog.lakekeeper.io-impl": "org.apache.iceberg.aws.s3.S3FileIO",
    "spark.sql.extensions": "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    "spark.sql.defaultCatalog": "lakekeeper",
    "spark.driver.memory": "24G",
    "spark.jars.packages": f"org.apache.iceberg:iceberg-spark-runtime-{SPARK_MINOR_VERSION}_2.13:{ICEBERG_VERSION},org.apache.iceberg:iceberg-aws-bundle:{ICEBERG_VERSION}",
    # about zstd: Turn on zstd to make things highly compressed. Less size on disk, less IO bandwidth!
    # we want to use zstd for parquet: 
    "spark.sql.parquet.compression.codec": "zstd",
    "spark.sql.iceberg.planning.preserve-data-grouping": "true",
    # we also want to use zstd for iceberg datafiles
    "spark.sql.iceberg.compression-codec": "zstd",
    "spark.sql.iceberg.locality.enabled": "true",
    # note: merge-schema should be only set to true if you "trust" the upstream data producer
    "spark.sql.iceberg.merge-schema": "false",
}

In [3]:
spark_config = SparkConf().setMaster('local[*]').setAppName("Iceberg-REST")
for k, v in config.items():
    spark_config = spark_config.set(k, v)

spark: SparkSession = SparkSession.builder.config(conf=spark_config).getOrCreate()


In [4]:
# spark.sql.shuffle.partitions (need to be smaller for streaming)
# the default is 200, which is great for huge data, but bad for small incremental streams
# spark.conf.get('spark.sql.shuffle.partitions')

In [5]:
spark.sql("USE lakekeeper")

DataFrame[]

In [6]:
# uncomment the following to view the full SparkConf
# spark.sparkContext.getConf().getAll()

In [1]:
if int(spark.conf.get("spark.sql.shuffle.partitions")) > 100:
    print("reducing shuffle partitions to 32")
    spark.conf.set("spark.sql.shuffle.partitions", "32")

NameError: name 'spark' is not defined

In [7]:
from pyspark.sql.functions import col, to_date, to_timestamp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType
schema = (StructType([
    StructField("event_time", StringType(), False),
    StructField("event_type", StringType(), False),
    StructField("product_id", IntegerType(), False),
    StructField("category_id", LongType(), False),
    StructField("category_code", StringType(), False),
    StructField("brand", StringType(), False),
    StructField("price", FloatType(), False),
    StructField("user_id", IntegerType(), False),
    StructField("user_session", StringType(), False),
]))

In [8]:
from pathlib import Path
dataset_dir = Path('/home/jovyan').joinpath('datasets').absolute()

In [9]:
ecomm_raw_dir = dataset_dir.joinpath('ecomm_raw')

In [10]:
october_data = (ecomm_raw_dir.joinpath("2019-Oct.csv")).as_posix()
november_data = (ecomm_raw_dir.joinpath("2019-Nov.csv")).as_posix()

## Create the E-commerce Data for ingestion to Iceberg
> we'll be using the '2019-Oct.csv', '2019-Nov.csv' datasets, doing some minor tweaks and then using these to populate our base Iceberg tables
> 
> Then we'll move on to doing Iceberg Streaming in Part 2

In [11]:
ecomm_oct_df = (
    spark.read.format("csv")
    .option("header", True)
    .schema(schema)
    .load(october_data)
)

In [12]:
ecomm_oct_df.show()

+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|2019-10-01 00:00:...|      view|  44600062|2103807459595387724|                NULL|shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|
|2019-10-01 00:00:...|      view|   3900821|2053013552326770905|appliances.enviro...|    aqua|   33.2|554748717|9333dfbd-b87a-470...|
|2019-10-01 00:00:...|      view|  17200506|2053013559792632471|furniture.living_...|    NULL|  543.1|519107250|566511c2-e2e3-422...|
|2019-10-01 00:00:...|      view|   1307067|2053013558920217191|  computers.notebook|  lenovo| 251.74|550050854|7c90fc70-0e80-459...|
|2019-10-01 00:00:...|      view|   1004237|205301355563188265

In [13]:
ecomm_oct_df.count()

42448764

In [14]:
ecomm_nov_df = (
    spark.read.format("csv")
    .option("header", True)
    .schema(schema)
    .load(november_data)
)

In [15]:
ecomm_nov_df.show()

+--------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand| price|  user_id|        user_session|
+--------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|2019-11-01 00:00:...|      view|   1003461|2053013555631882655|electronics.smart...|  xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|2019-11-01 00:00:...|      view|   5000088|2053013566100866035|appliances.sewing...|  janome|293.65|530496790|8e5f4f83-366c-4f7...|
|2019-11-01 00:00:...|      view|  17302664|2053013553853497655|                NULL|   creed| 28.31|561587266|755422e7-9040-477...|
|2019-11-01 00:00:...|      view|   3601530|2053013563810775923|appliances.kitche...|      lg|712.87|518085591|3bfb58cd-7892-48c...|
|2019-11-01 00:00:...|      view|   1004775|2053013555631882655|elect

In [16]:
ecomm_nov_df.count()

67501979

### Create the Initial Parquet Tables 
> These will live outside of MinIO and the Iceberg Catalog

* We'll create a simple helper function to _aid_ in our journey. We'll call it `write_parquet`.
* Using the new _function_, we'll then need to convert the **csv**->**parquet**, so run the **two** cells under "convert the data". They will produce raw parquet records under the path `datasets/ecomm_raw/parquet/ecomm`.
* We will then be able to run a series of 'daily' transactions to **write** all of the records into our foundational **iceberg** table.

In [17]:
def write_parquet(df: DataFrame, destination: Path, sink_dir: str) -> DataFrame:
    save_path = destination.joinpath('parquet', sink_dir)
    # modifications to the dataframe
    transformed = (
        df
            .withColumn("event_time", to_timestamp(col("event_time"), "yyyy-MM-dd HH:mm:ss z"))
            .withColumn("event_date", to_date(col("event_time")))
    )
    
    return (
        transformed
            .write
            .format("parquet")
            .partitionBy("event_date")
            .mode("append")
            .save(save_path.as_posix())
    )
    

### Convert the Data
We're on a mission to convert CSV to Parquet. Let's do that next.
> Note: This process may take a while if you're using the full dataset (~19GB)
> Note: The function "expects" that we'll be _appending__ to the **ecomm** directory.
> * If you want to modify the behavior of the function, give it a new argument called mode, and default that to "errorIfExists" of "ignore" - so you don't have to worry about deduplication or going nuculear and wiping out the entire ecomm directory!


In [18]:
# note: ecomm_raw_dir is the Path instance to the datasets/ecomm_raw directory
# further note: this could take a few minutes locally. Just turn up your favorite jams and let it ride
# lastly, the safe guard is on to check if the parquet directory already exists. This is to save you from accidentally 
# running the import more than once. This is to save you the trouble this can cause.

if not ecomm_raw_dir.joinpath('parquet', 'ecomm', 'event_date=2019-10-01').is_dir():
  write_parquet(ecomm_oct_df, ecomm_raw_dir, 'ecomm')
else:
  print("oh good. Saved you from having to import again. Not to mention the duplicates - what a mess")

oh good. Saved you from having to import again. Not to mention the duplicates - what a mess


In [19]:
if not ecomm_raw_dir.joinpath('parquet', 'ecomm', 'event_date=2019-11-01').is_dir():
  write_parquet(ecomm_nov_df, ecomm_raw_dir, 'ecomm')
else:
  print("same goes for the november set. It exists, so we'll skip for now.")

same goes for the november set. It exists, so we'll skip for now.


## Iceberg: Creating and Appending data to Iceberg Tables
> This section will help us build a firm understanding of how to work with Iceberg Tables
1. Learn to easily check for table existence
2. Learn to use Spark SQL to create Namespaces (this is where our tables live)
3. Use the simple table existence check to either a) create a new Iceberg table, or b) append new rows to an existing table within a Namespace

In [20]:
# let's identify a helper function to check to see if a "table" exists
# this can be done a few different ways, but this one uses the 'underlying' spark.catalog
# and I like that better :)

def table_exists(table_name: str):
    return any(table.name == table_name for table in spark.catalog.listTables())

## Tables live in Namespaces
Namespaces are a way of **grouping** tables together in the same way you would use **data domains** to separate different categories of data. 
> In practice, a namespace could be something like "consumer", "product", "orders" for ecommerce.
> For our demo, we'll just call the namespace `icystreams` since we are grouping data to use for _Streaming Iceberg_

In [21]:
catalog_namespace = 'icystreams'
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS {catalog_namespace}")
spark.sql("SHOW NAMESPACES").show()

+----------+
| namespace|
+----------+
|icystreams|
+----------+


In [22]:
# The current catalog will be the value of the following configuration: spark.conf.get("spark.sql.defaultCatalog")
spark.catalog.currentCatalog()

'lakekeeper'

In [23]:
# within the Apache Spark ecosystem. Our namespace is equivalent to a traditional hive "database", or Unity Catalog "schema"
spark.catalog.listDatabases()

[Database(name='icystreams', catalog='lakekeeper', description=None, locationUri='s3://examples/initial-warehouse/01998cb2-ac2e-7cb0-b341-ae5794445827')]

> Note: To simplify how we write to Iceberg, we are going to use the `spark.catalog.*` functions to point to `lakekeeper.icystreams`.

In [24]:
spark.catalog.setCurrentDatabase('icystreams')

In [ ]:
spark.catalog.listTables()

In [ ]:
spark.catalog.listTables()

In [28]:
def write_iceberg(df: DataFrame, namespace: str, table_name: str, partition_col: str) -> DataFrame:
    # simplifies the process of `testing` for the state of a given Iceberg table
    to_iceberg = (
        df
            .writeTo(f"{namespace}.{table_name}")
            .partitionedBy(partition_col)
    )
    # decided to append or create
    if table_exists(table_name):
        return to_iceberg.append()
    return to_iceberg.create()

In [71]:
# provide a way of creating hidden partitions
# note: check out all the tableproperties here (https://iceberg.apache.org/docs/latest/configuration/)
def write_iceberg_hidden_partitions(df: DataFrame, namespace: str, table_name: str, partitioned_by_cols: str) -> DataFrame:
    """
    This will create a new iceberg table (if it doesn't exist)
    :param df: The reference DataFrame to steal our schema from
    :param namespace: The namespace to write the table within
    :param table_name: The name of the table
    :param partitioned_by_cols: The PARTITIONED BY (clause). Example: 'days(event_time), category_id'
    :return: an empty DataFrame on success
    """
    # todo - add default ordering : ORDERED BY (category ASC, event_time DESC)
    if not table_exists(table_name):
        # fetch the ddl from the dataframe schema
        table_ddl = df.schema.toDDL()
        res_df = spark.sql(f"""
            CREATE TABLE IF NOT EXISTS {namespace}.{table_name} (
                {table_ddl}
            ) USING ICEBERG
            PARTITIONED BY ({partitioned_by_cols})
            TBLPROPERTIES(
            'read.split.planning-lookback'='5',
            'read.parquet.vectorization.batch-size'='32'
            );
        """)
        print(res_df.show())
    
    # since we explicitly create the table, we can simplify this vs the other version of the function
    return df.writeTo(f"{namespace}.{table_name}").append()
    

## Writing our Parquet into Iceberg
The following two cells will copy all data from the October and November 2019 parquet records
into the Iceberg tables `ecomm` and `ecomm_hidden`. 

The big difference here is that we don't need to "add" additional columns to the source parquet to this functionality.
> Note: If you are familiar with generated columns (generateAlwaysAs), then think of the PARTITIONED BY(days(event_time), bucket(8, category_id)) 
> as the same kind of mechanic. Remember, you can always change your mind later and drop or add additional partitoning without
> rewriting the entire table. Enjoy.


In [73]:
# set the reference to the parquet directory
parquet_dir = ecomm_raw_dir.joinpath('parquet', 'ecomm')
iceberg_table_name = 'ecomm'
iceberg_table_name_hp = "ecomm_hidden"

In [54]:
# Run the following block to write a single transaction per day
# Note: This took around 2 minutes on a high-powered M3 Ultra computer - 10 core (eg: spark.master=local[*])
# This can run on a single machine really easily - but blindingly fast in a cluster (only latency is IO between your object storage and your Iceberg Rest Catalog API and the Spark Cluster)
year = "2019"
months = ['10', '11']
for month in list(months):
    for day in (range(1, 32)) if month == '10' else (range(1, 31)):
        insert_date = f"{year}-{month}-0{day}" if day < 10 else f"{year}-{month}-{day}"
        write_iceberg(
            (spark
                .read
                .format('parquet')
                .load(parquet_dir.as_posix())
                .where(col("event_date").isin(insert_date))
            ),
        catalog_namespace,
        iceberg_table_name,
        'event_date'
        )


In [57]:
spark.sql(f"select count(*) as total from {catalog_namespace}.{iceberg_table_name}").show()

+---------+
|    total|
+---------+
|109950743|
+---------+


In [60]:
# shows a query that can do mix/max pushdown with simple partition by on 'event_date'
spark.sql(f"""
select
  product_id, 
  count(*) as total,
  min(event_time) as min_time, 
  max(event_time) as max_time 
  from {catalog_namespace}.{iceberg_table_name}
  where product_id in (17302664, 1003461) 
  and event_time BETWEEN TIMESTAMP('2019-10-01 18:00:00') and TIMESTAMP('2019-11-12 16:00:00') 
  group by product_id
""").show()

+----------+-----+-------------------+-------------------+
|product_id|total|           min_time|           max_time|
+----------+-----+-------------------+-------------------+
|  17302664| 2573|2019-10-01 18:00:17|2019-11-12 15:48:40|
|   1003461| 3911|2019-10-01 18:07:04|2019-11-12 15:56:11|
+----------+-----+-------------------+-------------------+


In [ ]:
# writing all the same data just with hidden partitioning
# note: there are only 691 categories, while we have over 200k unique product identifiers
# bucketing by product_id is another strategy to reduce the directory cost with hidden partitions
# try some things, break some things, it is fine :)
use_ddl_table = "ecomm_hidden_other"
year = "2019"
months = ['10', '11']
for month in list(months):
    for day in (range(1, 32)) if month == '10' else (range(1, 31)):
        insert_date = f"{year}-{month}-0{day}" if day < 10 else f"{year}-{month}-{day}"
        write_iceberg_hidden_partitions(
            df=(
                spark.read
                .format('parquet')
                .load(parquet_dir.as_posix())
                .where(col("event_date").isin(insert_date))
                .drop(col("event_date"))
            ),
            namespace=catalog_namespace,
            table_name=use_ddl_table,
            partitioned_by_cols='days(event_time), bucket(8, category_id)'
        )

In [75]:
hidden_parts_df = spark.sql(f"select * from {catalog_namespace}.{use_ddl_table}")
hidden_parts_df.show()

+-------------------+----------+----------+-------------------+--------------------+------------+-------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|       brand|  price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+------------+-------+---------+--------------------+
|2019-10-20 12:51:37|      view|  15200217|2053013553484398879|                NULL|        NULL| 123.56|545081263|c2e016df-d267-4cf...|
|2019-10-20 12:51:38|      view|   1801881|2053013554415534427|electronics.video.tv|     samsung| 493.93|514802508|ab915d15-8aae-49e...|
|2019-10-20 12:51:38|      view|   5801039|2053013553945772349|electronics.audio...|       hertz| 132.64|542448535|98dc10fa-427f-45f...|
|2019-10-20 12:51:38|  purchase|   5100742|2053013553341792533|  electronics.clocks|       honor| 148.01|517816925|6cba4ff0-0b07-4d8...|
|2019-10-20 12:51:39|      view|   180206

In [76]:
spark.sql(f"""
select
  product_id, 
  count(*) as total,
  min(event_time) as min_time, 
  max(event_time) as max_time 
  from {catalog_namespace}.{use_ddl_table}
  where product_id not in (17302664, 1003461)
  and event_time BETWEEN TIMESTAMP('2019-10-01 18:00:00') and TIMESTAMP('2019-11-12 16:00:00') 
  group by product_id
""").show()

+----------+-----+-------------------+-------------------+
|product_id|total|           min_time|           max_time|
+----------+-----+-------------------+-------------------+
|  49500003| 2054|2019-10-02 17:08:09|2019-11-01 18:21:43|
|  13200834| 9778|2019-10-01 18:21:21|2019-11-12 15:59:27|
|  49300028|  217|2019-10-02 00:51:05|2019-10-28 16:02:04|
|  17800204| 3519|2019-10-01 18:54:18|2019-11-12 15:57:24|
|   2700658|  452|2019-10-02 09:12:59|2019-11-12 15:08:43|
|  11500291| 1183|2019-10-01 18:19:03|2019-11-12 15:57:30|
|  13201247|  277|2019-10-02 06:06:14|2019-11-12 10:49:36|
|   2702332| 3362|2019-10-01 18:19:38|2019-11-12 15:58:09|
|   9100019|  285|2019-10-01 20:06:03|2019-11-12 15:33:22|
|   9101528|  174|2019-10-02 06:22:15|2019-11-12 05:55:10|
|  35108841|   13|2019-10-07 12:57:28|2019-11-08 14:50:27|
|  11500489|  170|2019-10-01 18:11:57|2019-11-12 06:34:15|
|   1600437|  228|2019-10-02 03:38:48|2019-11-12 08:23:19|
|   6100213|  208|2019-10-02 10:42:16|2019-11-12 14:49:2

In [80]:
spark.sql(f"show create table {catalog_namespace}.{use_ddl_table}").show(100, 0, True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 createtab_stmt | CREATE TABLE lakekeeper.icystreams.ecomm_hidden_other (\n  event_time TIMESTAMP,\n  event_type STRING,\n  product_id INT,\n  category_id BIGINT,\n  category_code STRING,\n  brand STRING,\n  price FLOAT,\n  user_id INT,\n  user_session STRING)\nUSING iceberg\nPARTITIONED BY (days(event_t

In [56]:
spark.sql(f"select count(*) as total from {catalog_namespace}.{iceberg_table_name_hp}").show()

+---------+
|    total|
+---------+
|109950743|
+---------+


In [66]:
spark.sql(f"select distinct(category_id) as category_id from {catalog_namespace}.{iceberg_table_name_hp}").toPandas()

,category_id
0,2060237588744111062
1,2090971686529663114
2,2053013564003713919
3,2053013556344914381
4,2152167773222993940
...,...
686,2100064855133258156
687,2053013560111399597
688,2053013552259662037
689,2055156924273394455


In [64]:
# Note: we've got 206,876 unique product ids. This would be a bad column to partition on
spark.sql(f"select count(distinct(product_id)) as unique_product_ids from {catalog_namespace}.{iceberg_table_name_hp}").show()

+------------------+
|unique_product_ids|
+------------------+
|            206876|
+------------------+


In [65]:
# Note: the cardinality of the category id's is only 691 distinct
spark.sql(f"select count(distinct(category_id)) as unique_category_ids from {catalog_namespace}.{iceberg_table_name_hp}").show()

+-------------------+
|unique_category_ids|
+-------------------+
|                691|
+-------------------+


In [67]:
spark.sql(f"select distinct(product_id) as product_id from {catalog_namespace}.{iceberg_table_name_hp}").toPandas()

,product_id
0,28300055
1,1307545
2,38900025
3,5500325
4,4900373
...,...
206871,100025269
206872,26006635
206873,24300264
206874,10800003


In [86]:
# shows a query that can do mix/max pushdown and leans into the hidden date partitions to do file skipping
# you'll notice that we can do arbitrary filtering by hour as well.
# this query isn't taking advantage of category filtering (see how the two tables compare as homework)
spark.sql(f"""
select
  product_id, 
  count(*) as total,
  min(event_time) as min_time, 
  max(event_time) as max_time 
  from {catalog_namespace}.{iceberg_table_name_hp}
  where product_id not in (17302664, 1003461)
  and event_time BETWEEN TIMESTAMP('2019-10-01 18:00:00') and TIMESTAMP('2019-11-12 16:00:00') 
  group by product_id
""").show()

+----------+-----+-------------------+-------------------+
|product_id|total|           min_time|           max_time|
+----------+-----+-------------------+-------------------+
|  28715821|  777|2019-10-02 06:59:28|2019-11-12 06:09:49|
|  28706430| 2589|2019-10-02 00:35:31|2019-11-03 15:06:39|
|  15600022| 1089|2019-10-01 18:18:07|2019-11-12 14:02:58|
|  26600154|  148|2019-10-02 12:51:58|2019-11-12 15:36:18|
|   4300425|  109|2019-10-02 06:36:54|2019-11-12 13:07:04|
|  28703068|   76|2019-10-12 01:43:51|2019-11-12 15:30:56|
|  47000059|  455|2019-10-04 05:54:34|2019-11-12 15:19:28|
|  30000077|  758|2019-10-01 23:12:23|2019-11-12 14:48:08|
|  22200112|  173|2019-10-01 19:52:42|2019-11-12 12:11:30|
|  52900142|  182|2019-10-10 02:51:48|2019-11-12 15:28:07|
|  45600191|  267|2019-10-16 11:24:10|2019-11-10 15:19:18|
|   4300232|  158|2019-10-09 13:53:21|2019-11-01 05:46:16|
|   1480506| 1477|2019-10-01 18:10:39|2019-11-12 03:55:18|
|  17100091|   89|2019-10-01 18:08:59|2019-11-12 11:27:1

In [70]:
# view the create table statement
spark.sql(f"show create table {catalog_namespace}.{iceberg_table_name}").toPandas()

,createtab_stmt
0,CREATE TABLE lakekeeper.icystreams.ecomm (\n ...


In [69]:
spark.sql(f"describe extended {catalog_namespace}.{iceberg_table_name}").toPandas()

,col_name,data_type,comment
0,event_time,timestamp,None
1,event_type,string,None
2,product_id,int,None
3,category_id,bigint,None
4,category_code,string,None
5,brand,string,None
6,price,float,None
7,user_id,int,None
8,user_session,string,None
9,event_date,date,None


In [27]:
spark.sql(f"select count(*) as total from {catalog_namespace}.{iceberg_table_name}").toPandas()

,total
0,109950743


In [78]:
spark.sql(f"""
select * from {catalog_namespace}.{iceberg_table_name} 
where event_date BETWEEN DATE("2019-11-15") AND DATE("2019-11-30")
ORDER BY event_date DESC
LIMIT 15
""").toPandas()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,event_date
0,2019-11-30 00:00:00,view,4804055,2053013554658804075,electronics.audio.headphone,apple,196.270004,548106089,e0b70673-d26d-4c72-98e8-7c25bb8d1216,2019-11-30
1,2019-11-30 00:00:00,view,1004225,2053013555631882655,electronics.smartphone,apple,952.150024,579329479,e302bdd1-1f8c-4210-9920-a55036d2e8c9,2019-11-30
2,2019-11-30 00:00:00,view,12703410,2053013553559896355,None,cordiant,47.619999,574179352,69e96ac7-8042-42bf-b145-40bc937dd141,2019-11-30
3,2019-11-30 00:00:01,view,3601438,2053013563810775923,appliances.kitchen.washer,beko,215.889999,555019938,626534f8-eb2a-4d05-84d4-9b9d91edb9d8,2019-11-30
4,2019-11-30 00:00:02,view,6400179,2053013554121933129,computers.components.cpu,intel,203.869995,513342746,4ba0eec3-1026-4df4-9495-17773a86ead5,2019-11-30
5,2019-11-30 00:00:02,view,18400149,2053013553912217915,None,kicx,112.489998,521867315,824aa67e-7220-4735-9507-0f37b3d5c27b,2019-11-30
6,2019-11-30 00:00:02,view,40900011,2127425434894205468,construction.tools.painting,None,48.910000,569407672,5d9b3bbd-4efc-4b7c-86dd-8cb9e2bc8f77,2019-11-30
7,2019-11-30 00:00:02,view,3500093,2053013555287949705,None,None,33.459999,553748453,4310c6c8-e10c-49db-a85c-051aaf0969d3,2019-11-30
8,2019-11-30 00:00:03,view,6000167,2053013560807654091,auto.accessories.alarm,centurion,82.110001,547094018,cea57a4d-5343-4de2-81fc-48124020b8bf,2019-11-30
9,2019-11-30 00:00:03,view,1004225,2053013555631882655,electronics.smartphone,apple,952.150024,579329479,e302bdd1-1f8c-4210-9920-a55036d2e8c9,2019-11-30


## Is the Query Slow or is it Just your Laptop?
These queries are super fast. But we're also not streaming yet. 
* We haven't optimized any of the tables at this point.
* The next few cells show how to use the Iceberg stored procedures.  
* Whether this is a local experiment or production, you'll still need to periodically "clean" your tables up!

In [81]:
#result_df = spark.sql(f"""
#CALL system.rewrite_data_files(
#    table => 'lakekeeper.{catalog_namespace}.{iceberg_table_name}',
#        options => map(
#            'target-file-size-bytes', '134217728', -- 128MB target size
#            'min-input-files', '5' -- Minimum files to trigger rewrite
#        )
#    );
#""")

# result_df = spark.sql(f"CALL system.rewrite_data_files(table => 'lakekeeper.{catalog_namespace}.{iceberg_table_name}')")

# expire snapshots and remove manifest lists
result_df = spark.sql(f"""
CALL system.expire_snapshots(
  table => 'lakekeeper.{catalog_namespace}.{iceberg_table_name}', 
  older_than => '2025-09-28 00:00:00'
)
""")

In [82]:
result_df.show(1, 0, True)

-RECORD 0----------------------------------
 deleted_data_files_count            | 0   
 deleted_position_delete_files_count | 0   
 deleted_equality_delete_files_count | 0   
 deleted_manifest_files_count        | 0   
 deleted_manifest_lists_count        | 60  
 deleted_statistics_files_count      | 0   


In [83]:
result_df = spark.sql(f"""
CALL system.expire_snapshots(
  table => 'lakekeeper.{catalog_namespace}.{iceberg_table_name_hp}', 
  older_than => '2025-09-28 00:00:00'
)
""")
result_df.show(1, 0, True)

NameError: name 'results_df' is not defined

In [85]:
result_df = spark.sql(f"""
CALL system.expire_snapshots(
  table => 'lakekeeper.{catalog_namespace}.{use_ddl_table}', 
  older_than => '2025-09-28 00:00:00'
)
""")
result_df.show(1, 0, True)

-RECORD 0----------------------------------
 deleted_data_files_count            | 0   
 deleted_position_delete_files_count | 0   
 deleted_equality_delete_files_count | 0   
 deleted_manifest_files_count        | 0   
 deleted_manifest_lists_count        | 60  
 deleted_statistics_files_count      | 0   


## Fruits of our Labor?
Go ahead and **run the SQL query from before we tried to optimize the table** again. You may notice we're not getting a boost in speed. So what is happening here? 

The tables are actually performing really well. We don't have hundreds of gigabytes of metadata stacking up. The maintenance tasks so far are not a problem.

* Next, we're going to move on and utilize our **Foundational Tables** as the basis for our Streaming Iceberg best practices. Follow along :) 

In [32]:
# > Note: This fails with S3 IO.
#clear_orphans_df = spark.sql(f"""
#CALL system.remove_orphan_files(
#  table => 'lakekeeper.{catalog_namespace}.{iceberg_table_name}',
#  prefix_listing => false
#)
#""")
# clear_orphans_df.show(1, 0, True)

In [ ]:
# > Note: This is the nuclear option to scrap the table and all metadata and simply begin again!
# spark.sql(f"DROP TABLE {catalog_namespace}.{iceberg_table_name}")
#spark.sql(f"DROP NAMESPACE {catalog_namespace}")